# Méthodes d'interprétabilité pour les modèles de Machine Learning

## Import des librairies

In [2]:
# Packages
import numpy as np
from pygam import LogisticGAM, s, l
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import shap
import matplotlib.pyplot as plt
np.random.seed(2026)
import warnings
warnings.filterwarnings("ignore")

## Import des données

Le jeu de données Adult permet d'entraîner des modèles de classification binaire, puisque l'objectif est de prédire si un individu de la base de données gagne un revenu annuel supérieur à 50K dollars ($Y=1$) ou non ($Y=0$). Les données proviennent du recensement américain (U.S. Census), et sont disponibles via la base UCI Machine Learning Repository : https://archive.ics.uci.edu/dataset/2/adult.

Le jeu de données Adult contient des informations socio-démographiques, comme par exemple :
- l'âge,
- le niveau d’éducation,
- le statut marital,
- la profession de l'individu,
- les heures travaillées par semaine
- le gain en capital et la perte en capital,
- le pays d’origine de l'individu,
- etc.

In [3]:
# Chargement des données : https://archive.ics.uci.edu/dataset/2/adult
# Colonnes utilisées pour le modèle de classification
cols = [
    "age", "workclass", "fnlwgt", "education", "education_num",
    "marital_status", "occupation", "relationship", "race", "sex",
    "capital_gain", "capital_loss", "hours_per_week", "native_country",
    "income"
]

adult = pd.read_csv("data/adult/adult.data", names=cols, na_values=" ?", skipinitialspace=True)

Combien y a-t-il de lignes avec des valeurs manquantes ?

In [ ]:
rows_with_nan = ...
print(rows_with_nan)

In [ ]:
# Suppression des lignes avec des valeurs manquantes
adult = ...

In [ ]:
# Nombre d'observations et nombre de colonnes dans le jeu de données
n_obs, n_cols = ...
print(f"Le jeu de données Adult contient {n_obs} observations et {n_cols} variables, dont la variable cible 'income'.")

In [ ]:
# Observation des 5 premières lignes du jeu de données
adult[:5]

In [ ]:
# Affiche les types des différentes variables de Adult
print("Voici le type des différentes variables du jeu de données (catégorielles ou numériques) :")
print(...)

In [ ]:
# Exemple de variable de type "object" : variable catégorielle
np.unique(adult["workclass"])

Quelles sont les fréquences empiriques des différentes modalités de la variable à prédire $Y$ ?

In [ ]:
print("Voici la fréquence de la variable à prédire 'income' :")
print(...)

On va transformer la variable cible 'income' en variable binaire 0/1.

In [ ]:
# Variable cible (ou variable à prédire)
print(adult["income"][:10].to_list())

y = (adult["income"] == ">50K").astype(int).values
print(y[:10].tolist())


On utilisera seulement quelques variables pour entraîner le modèle de classification :
- l'âge,
- le niveau d'éducation,
- le nombre d'heures travaillées par semaine, 
- le gain et la perte en capital de l'individu,
- et le sexe de l'individu.

In [ ]:
# Sélection de variables
X_df = adult[
    ["age", "education_num", "hours_per_week", "capital_gain", "capital_loss", "sex"]
].copy()

X_df[:5]

On va encoder la variabele 'sex' initialement catégorielle en variable numérique.

In [ ]:
# Encodage du sexe (one-hot encoding)
X_df["sex"] = (X_df["sex"] == "Male").astype(int)
X_df[:5]

La catégorie 'Female' devient la catégorie de référence (valeur à 0) et la catégorie 'Male' prend la valeur 1.

On peut ensuite transformer le dataframe pandas des variables explicatives en array numpy, pour faciliter l'entraînement du modèle GAM par la suite, avec les librairies classiques de Python.

In [ ]:
X = X_df.values
X[:5]

On peut désormais séparer le jeu de données en une base d'entraînement (70% des observations) et une base de test (30% des observations).

In [ ]:
# Séparation en jeu de données d'entraînement et de test
X_train, X_test, y_train, y_test = ... # utilise random_state = 42

## Exemple de modèle interprétable par nature : le GAM

Ici, comme la variable à prédire est binaire, on considérera un GAM Logistique.

### Premier modèle : GAM univarié

Considérons le premier modèle simplifié n'utilisant que la variable explicative "age". On cherche à estimer :
$$\mathbb{P}(Y=1 \mid \text{age}) = \sigma\!\Big(\beta_0
+ f_1(\text{age})
\Big) \enspace ,$$

où $\sigma$ correspond à la fonction sigmoide :
$$
\forall t \in \mathbb{R}, \enspace \sigma(t)=\frac{1}{1+e^{-t}} \in [0,1] \enspace .
$$

La fonction inverse de la fonction sigmoide est la fonction logit :
$$
\forall p \in ]0,1[, \enspace \text{logit}(p) = \log\left(\frac{p}{1-p}\right) \in \mathbb{R} \enspace .
$$

Pour modéliser une spline sur Python, on utilise la fonction $s(.)$, qui correspond à une spline de lissage, notée $f_1$ ci-dessus. En pratique, il n'y a pas autant de nœuds que de points de données dans la base d'apprentissage, sinon le calcul serait trop complexe.

Dans la fonction $s(.)$ :
- le premier terme correspond au numéro de la colonne des variables explicatives de $\mathbf{X}_{\text{train}}$ que l'on souhaite modéliser par une spline (ici, la variable "age"),
- "spline_order" correspond à l'ordre des polynômes (par morceaux) qui vont être fittés localement,
- "n_splines" correspond au nombre de fonctions choisi pour représenter $s(\text{age}) = \sum_{j=1}^{\text{n\_splines}} \beta_{j} \cdot b_{j}(\text{age})$,
- "lam" est la valeur du paramètre de lissage $\lambda$.

Applique le modèle GAM Logistique présenté ci-dessus avec une spline qui contient 3 fonctions de base et des polynômes de degré 1.

In [ ]:
# Définition du modèle
gam = LogisticGAM(
    s(0, spline_order = ..., n_splines = ..., lam = .0001) # Terme de spline pour la variable âge
)

In [ ]:
# Entraînement du modèle
gam.fit(X_train, y_train)

In [ ]:
# On peut observer l'effet de la variable "age" sur la variable à prédire "income"
feature_name = "age"
feature_index = 0
# Effet partiel de la variable "age" sur les données observées
plt.plot(gam.partial_dependence(term=feature_index))

plt.title(f"Effet de {feature_name}")
plt.xlabel(feature_name)
plt.ylabel(r"Effet partiel sur logit($\mathbb{P}(Y=1)$)")
plt.show()

On peut récupérer les 3 coefficients de la fonction $s(.)$ : $\beta_0$, $\beta_1$ et $\beta_2$.

In [ ]:
cpt_coef = 0
for i, term in enumerate(gam.terms):
    # On ignore l'intercept
    if term.isintercept:
        continue
    n_coefs = term.n_coefs
    print("Nombre de fonctions de base dans la spline : ", n_coefs)
    start = i + cpt_coef
    cpt_coef += n_coefs
    end = i + cpt_coef
    coefs = gam.coef_[start:end]
    print(f"Coefficients pour le terme {i} :\n {coefs}")

On peut également récupérer les valeurs des différentes fonctions de base sur les points des données d'entraînement, $b_k(\text{age}_i)$, $i \in \{1,\cdots,n\}$.

In [ ]:
# Matrice du modèle pour s(0)
B_matrix = gam._modelmat(X_train, term=feature_index)
print(np.shape(B_matrix))

Chaque ligne correspond à une observation (une ligne de $\mathbf{X}_{\text{train}}$).  
Chaque colonne correspond à une fonction de base $b_k(\cdot)$.  

Ainsi, la matrice de design $B_{\text{matrix}}$ s'écrit :

$$
B_{\text{matrix}}[i,k] = b_k(\text{age}_i)
$$

où :  
- $\text{age}_i$ est la valeur de la variable "age" pour l'observation $i$,  
- $b_k$ est la $k$-ième fonction de base de la spline.

"B_matrix @ coefs_term" donne directement les valeurs de la fonction spline $s(\text{age})$ pour toutes les observations ($n = 22792$) :

In [ ]:
B_array = B_matrix.toarray()
s_data = B_array @ coefs  # valeur de la spline pour chaque observation
print(np.shape(s_data))
print(s_data[:10])

In [ ]:
# On peut tracer la spline obtenue pour la variable âge
for term_idx, term in enumerate(gam.terms):
    # On ignore le terme intercept
    if term.isintercept:
        continue  

    # Indices des coefficients pour ce terme
    start = sum(t.n_coefs for t in gam.terms[:term_idx])
    end = start + term.n_coefs
    coefs = gam.coef_[start:end]

    # Créer une grille pour la variable "age"
    x_vals = np.linspace(X_train[:, term_idx].min(), X_train[:, term_idx].max(), 200)
    
    # On crée une grille pour la variable "age" (les autres variables sont mises à leur moyenne)
    X_grid = np.tile(X_train.mean(axis=0), (200,1))
    X_grid[:, term_idx] = x_vals

    # Matrice de B-splines pour ce terme
    B_grid = gam._modelmat(X_grid, term=term_idx).toarray()

    # Fonction spline reconstruite
    s_grid = B_grid @ coefs

    # --- Tracer ---
    plt.figure(figsize=(6,4))
    plt.plot(x_vals, s_grid, color='blue', label='Spline estimée')
    # Tracer chaque spline de base (b_j)
    for k in range(B_grid.shape[1]):
        plt.plot(x_vals, B_grid[:,k], linestyle='--', alpha=0.5, label=f'b{k}' if k<=5 else "")
    plt.scatter(X_train[:, term_idx], s_data, s=5, alpha=0.3, color='red', label='Observations')
    plt.title(f"Spline pour {feature_name}")
    plt.xlabel(feature_name)
    plt.ylabel(r"Effet partiel sur logit($\mathbb{P}(Y=1)$)")
    plt.legend()
    plt.tight_layout()
    plt.show()

On peut faire varier la valeur de $\lambda$ pour voir l'effet sur la spline obtenue.

In [ ]:
# Valeurs de lambda à tester
lambda_values = [0.01, 0.5, 3, 100]

# Couleurs pour chaque courbe
colors = ['blue', 'green', 'orange', 'red']

# Indices et noms
feature_name = "age"
feature_index = 0

for lam, color in zip(lambda_values, colors):
    # Définition du modèle avec la lambda actuelle
    gam = LogisticGAM(
        s(feature_index, spline_order=2, n_splines=3, lam=...)
    )
    
    # Ajustement du modèle sur les données
    gam.fit(X_train, y_train)
    
    # Tracer la spline estimée (effet partiel de la variable "age")
    plt.plot(gam.partial_dependence(term=feature_index), color=color, label=f'lambda={lam}')
    
plt.title(fr"Effet partiel de {feature_name} pour différentes valeurs de $\lambda$")
plt.xlabel(feature_name)
plt.ylabel(r"Effet partiel sur logit($\mathbb{P}(Y=1)$)")
plt.legend()
plt.show()

On peut également changer l'ordre des polynômes par morceaux et le nombre de splines, et observer l'effet obtenu sur la fonction s(.) fittée.

In [ ]:
# Définition du modèle
gam = LogisticGAM(
    s(0, spline_order = ..., n_splines = ..., lam = .0001) # Terme de spline pour la variable âge
)
# Entraînement du modèle
gam.fit(X_train, y_train)
# On peut observer l'effet de la variable âge sur la variable à prédire 'income'
feature_name = "age"
feature_index = 0
# Effet partiel de la variable "age" sur les données observées
plt.plot(gam.partial_dependence(term=feature_index))

plt.title(f"Effet de {feature_name}")
plt.xlabel(feature_name)
plt.ylabel(r"Effet partiel sur logit($\mathbb{P}(Y=1)$)")
plt.show()

In [ ]:
# On peut tracer la spline obtenue pour la variable âge
for term_idx, term in enumerate(gam.terms):
    # On ignore le terme intercept
    if term.isintercept:
        continue  

    # Indices des coefficients pour ce terme
    start = sum(t.n_coefs for t in gam.terms[:term_idx])
    end = start + term.n_coefs
    coefs = gam.coef_[start:end]

    # Créer une grille pour la variable "age"
    x_vals = np.linspace(X_train[:, term_idx].min(), X_train[:, term_idx].max(), 200)
    
    # On crée une grille pour la variable "age" (les autres variables sont mises à leur moyenne)
    X_grid = np.tile(X_train.mean(axis=0), (200,1))
    X_grid[:, term_idx] = x_vals

    # Matrice de B-splines pour ce terme
    B_grid = gam._modelmat(X_grid, term=term_idx).toarray()

    # Fonction spline reconstruite
    s_grid = B_grid @ coefs

    # --- Tracer ---
    plt.figure(figsize=(6,4))
    plt.plot(x_vals, s_grid, color='blue', label='Spline estimée')
    # Tracer chaque spline de base (b_j)
    for k in range(B_grid.shape[1]):
        plt.plot(x_vals, B_grid[:,k], linestyle='--', alpha=0.5, label=f'b{k}' if k<=5 else "")
    plt.scatter(X_train[:, term_idx], s_data, s=5, alpha=0.3, color='red', label='Observations')
    plt.title(f"Spline pour {feature_name}")
    plt.xlabel(feature_name)
    plt.ylabel(r"Effet partiel sur logit($\mathbb{P}(Y=1)$)")
    plt.legend()
    plt.tight_layout()
    plt.show()

### Deuxième modèle GAM multivarié : plusieurs variables explicatives

Considérons le deuxième modèle utilisant toutes les variable explicatives. On cherche à estimer :
$$\mathbb{P}(Y=1 \mid \mathbf{X}) = \sigma\!\Big(\beta_0
+ f_1(\text{age})
+ f_2(\text{education})
+ f_3(\text{hours})
+ f_4(\text{capital\_gain})
+ f_5(\text{capital\_loss})
+ \beta_s \,\text{sex}
\Big) \enspace .$$ 

In [ ]:
# Noms des colonnes
feature_names = list(X_df.columns)
print(feature_names)

In [ ]:
# Définition du modèle
gam = LogisticGAM(
    s(0) +    # spline avec les paramètres par défaut pour la variable "age"
    ...  +    # spline avec les paramètres par défaut pour la variable "education_num"
    ...  +    # spline avec les paramètres par défaut pour la variable "hours_per_week" 
    ...  +    # spline avec les paramètres par défaut pour la variable "capital_gain" 
    ...  +    # spline avec les paramètres par défaut pour la variable "capital_loss" 
    l(5)      # terme linéaire pour la variable "sex"
)

On peut trouver la valeur optimale de $\lambda$ par validation croisée (5-folds par défaut).

In [ ]:
# Algorithme de validation croisée pour trouver les lambdas optimaux pour la log-vraisemblance
gam.gridsearch(...)

  0% (0 of 11) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--
  9% (1 of 11) |##                       | Elapsed Time: 0:00:00 ETA:   0:00:08
 18% (2 of 11) |####                     | Elapsed Time: 0:00:01 ETA:   0:00:05
 27% (3 of 11) |######                   | Elapsed Time: 0:00:01 ETA:   0:00:04
 36% (4 of 11) |#########                | Elapsed Time: 0:00:01 ETA:   0:00:03
 45% (5 of 11) |###########              | Elapsed Time: 0:00:02 ETA:   0:00:02
 54% (6 of 11) |#############            | Elapsed Time: 0:00:02 ETA:   0:00:02
 63% (7 of 11) |###############          | Elapsed Time: 0:00:02 ETA:   0:00:01
 72% (8 of 11) |##################       | Elapsed Time: 0:00:03 ETA:   0:00:01
 81% (9 of 11) |####################     | Elapsed Time: 0:00:03 ETA:   0:00:00
 90% (10 of 11) |#####################   | Elapsed Time: 0:00:03 ETA:   0:00:00
100% (11 of 11) |########################| Elapsed Time: 0:00:04 Time:  0:00:04


LogisticGAM(callbacks=[Deviance(), Diffs(), Accuracy()], 
   fit_intercept=True, max_iter=100, 
   terms=s(0) + s(1) + s(2) + s(3) + s(4) + l(5) + intercept, 
   tol=0.0001, verbose=False)

On obtient un $\lambda$ optimal par variable explicative, même pour le terme linéaire de la variable "sex" (pénalisation Ridge).

In [ ]:
# Affiche les différents lambdas obtenus
print("Lambda optimal :", ...)
# Affiche le nombre de lambdas obtenus
print("Nombre de lambdas : ", ...)

Lambda optimal : [[np.float64(0.001)], [np.float64(0.001)], [np.float64(0.001)], [np.float64(0.001)], [np.float64(0.001)], [np.float64(0.001)]]
Nombre de lambdas :  6


On peut tracer les effets des différentes variables explicatives du modèle GAM sur logit($\mathbb{P}(Y=1)$).

In [ ]:
plt.figure(figsize=(10, 6))

for i in range(6):
    plt.subplot(2, 3, i+1)  # 2 lignes x 3 colonnes
    
    # Effet partiel sur les données observées
    pdep, confi = gam.partial_dependence(term=i, X=X_train, width=0.95)
    
    # Tracer l’effet partiel
    #plt.plot(X_train[:, i], pdep, 'o', alpha=0.3, label='Effet partiel')
    plt.plot(gam.partial_dependence(term=i))
    
    #plt.title(f"Effet de {feature_names[i]}")
    plt.xlabel(feature_names[i])
    plt.ylabel(r"Effet partiel sur logit($\mathbb{P}(Y=1)$)")
    
plt.tight_layout()
plt.show()

On peut afficher les coefficients $\beta_{j,k}$, des différentes variables explicatives $X_j$ :

In [ ]:
cpt_coef = 0
for i, term in enumerate(gam.terms):
    if term.isintercept:
        continue
    if i == 5:
        continue
    n_coefs = term.n_coefs
    #print(i, n_coefs)
    start = i + cpt_coef
    cpt_coef += n_coefs
    end = i + cpt_coef
    #print(start, end)
    coefs = gam.coef_[start:end]
    print(f"Coefficients pour le terme de spline {i} : {coefs}")

Quel est le nombre de fonctions de base "n_splines" utilisé par défaut par scikit-learn dans la fonction $s(.)$ ?

In [ ]:
...

On peut tracer à nouveau le terme de spline de la variable "age".

In [ ]:
for term_idx, term in enumerate(gam.terms):
    if term.isintercept:
        continue  # ignorer le terme constant
    
    if term_idx == 0:
        # Indices des coefficients pour ce terme
        start = sum(t.n_coefs for t in gam.terms[:term_idx])
        end = start + term.n_coefs
        coefs = gam.coef_[start:end]

        # Créer une grille pour la feature
        x_vals = np.linspace(X_train[:, term_idx].min(), X_train[:, term_idx].max(), 200)
        
        # Créer X complet pour _modelmat (les autres features = moyenne)
        X_grid = np.tile(X_train.mean(axis=0), (200,1))
        X_grid[:, term_idx] = x_vals

        # Matrice de B-splines pour ce terme
        B_grid = gam._modelmat(X_grid, term=term_idx).toarray()

        # Fonction spline reconstruite
        f_grid = B_grid @ coefs

        # --- Tracer ---
        plt.figure(figsize=(6,4))
        plt.plot(x_vals, f_grid, color='blue', label='Spline estimée')
        # Tracer chaque B-spline de base
        for k in range(B_grid.shape[1]):
            plt.plot(x_vals, B_grid[:,k], linestyle='--', alpha=0.5, label=f'b{k}' if k<=5 else "")
        plt.title(f"Spline pour {feature_names[term_idx]}")
        plt.xlabel(feature_names[term_idx])
        plt.ylabel(r"Effet partiel sur logit($\mathbb{P}(Y=1)$)")
        plt.legend()
        plt.tight_layout()
        plt.show()


On va désormais calculer deux métriques de performance prédictive du modèle GAM.

In [ ]:
# Prédictions sur la base d'entraînement
y_proba_train = ...
y_pred_train = ...

# Prédictions sur la base de test
y_proba_test = ...
y_pred_test = ...

Sur la base d'entraînement :

In [ ]:
# Accuracy
acc = ...(y_train, y_pred_train)
# AUC
auc = ...(y_train, y_proba_train)

print(f"Accuracy (train) : {acc:.3f}")
print(f"AUC (train)      : {auc:.3f}")

Sur la base de test :

In [ ]:
acc = ...(y_test, y_pred_test)
auc = ...(y_test, y_proba_test)

print(f"Accuracy (test) : {acc:.3f}")
print(f"AUC (test)      : {auc:.3f}")

## Modèle de classification non interprétable : Forêt Aléatoire

In [ ]:
# Séparation de la base Adult en bases d'entraînement et de test
X_df_train, X_df_test, y_train, y_test = train_test_split(
    X_df, y,
    test_size=0.3,
    random_state=42
)

In [ ]:
# Définition du modèle
rf = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)

A quoi correspondent les paramètres "n_estimators" et "max_depth" ?

...

In [ ]:
# Entraînement du modèle
...

# Prédictions du modèle sur la base d'entraînement
y_pred_train = ...
y_proba_train = ...
# Prédictions du modèle sur la base de test
y_pred_test = ...
y_proba_test = ...

# Métriques de performance sur la base de test : accuracy et AUC
print("Accuracy:", accuracy_score(y_test, y_pred_test))
print("AUC:", roc_auc_score(y_test, y_proba_test))

Accuracy: 0.8448152318558706
AUC: 0.8749960726618426


La fonction de scikit-learn "RandomForestClassifier" contient un vecteur associé pour quantifier l'importance des variables explicatives.
Pour chaque variable : 
- on fait la somme des réductions de l'indice de Gini (utilisé pour splitter les noeuds de façon optimale pour chaque arbre composant la forêt) à tous les noeuds où elle est utilisée,
- on fait la moyenne de cette somme sur tous les arbres de la forêt,
- puis, on normalise sur toutes les variables explicatives pour que la somme des importances fassent 1.

In [ ]:
# Suppose rf_model est ton Random Forest déjà entraîné
feature_importances = ...  # array de taille n_features

# Mettre en DataFrame pour plus de lisibilité
df_importance = pd.DataFrame({
    "feature": feature_names,
    "importance": feature_importances
}).sort_values(by="importance", ascending=False)

print(df_importance)

In [ ]:
# Calcule la somme de l'importance des différentes variables
...

## Partial Dependence Plot (PDP) pour la Forêt Aléatoire

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

Si on note $\hat{f}$ le modèle estimé par la Forêt Aléatoire, on peut calculer le PDP de la variable "age" à la valeur $z$ :
$$
\widehat{\text{PD}}_{\text{age}}(z)=\frac{1}{n}\sum_{i=1}^n f(\text{z}, \mathbf{x}_{i,-\text{age}}) \enspace ,
$$

In [ ]:
# Exemple pour la variable 'age'
PartialDependenceDisplay.from_estimator(
    ...,              # modèle entraîné
    ...,         # données d'entraînement
    features=['age'], # feature à analyser
    kind='average',  # moyenne marginale (PDP classique)
    grid_resolution=20 # on utilise 20 valeurs de la variable "age" pour calculer son PDP
)
plt.show()

On peut aussi spécifier à la fonction plusieurs variables à la fois pour lesquelles on veut calculer le PDP.

In [ ]:
PartialDependenceDisplay.from_estimator(
    ...,           # modèle entraîné
    ...,            # données d'entraînement
    features=..., # liste des variables pour le PDP : "age" et "hours_per_week"
    kind="average",     # "average" = PDP classique
    grid_resolution=20  # nombre de points pour la grille
)

plt.show()

## Méthode agnostique et locale : SHAP

"TreeExplainer" est l’explainer TreeSHAP codé sur le package Python SHAP, optimisé pour les modèles basés sur des arbres comme les :
- Random Forest,
- Gradient Boosting (XGBoost, LightGBM, CatBoost),
- Decision Tree.

"TreeExplainer" est très rapide pour les arbres, contrairement à "KernelExplainer" qui s'applique à n'importe quel modèle de machine learning mais qui est généralement plus lent.

In [ ]:
# TreeSHAP pour Random Forest
explainer_shap = ...(rf)
# Récupération des valeurs SHAP pour la base de données de test
shap_values = explainer_shap.shap_values(...)

In [ ]:
print(np.shape(shap_values))   # (n_samples, n_features, 2) car problème de classification binaire
print(shap_values[0].shape)    # (n_samples, n_features)
print(shap_values[1].shape)    # (n_samples, n_features)
print(X_df_test.shape)         # (n_samples, n_features)

On peut afficher les valeurs de Shapley pour la variable "age" (première variable explicative du modèle), notées $\phi_1$, pour chacune des observations de l'ensemble de test. 

Deux valeurs de Shapley sont disponibles par variable et par observation : pour les estimations de $\mathbb{P}(Y=0)$ et $\mathbb{P}(Y=1)$. Pour une observation donnée, les deux $\phi_1$ qui sont symétriques car $\mathbb{P}(Y=0) = 1-\mathbb{P}(Y=1)$.

In [ ]:
# Affichage des valeurs SHAP pour la variable "age" et les 5 premières observations pour Y=0
print(shap_values[...])
# Affichage des valeurs SHAP pour la variable "age" et les 5 premières observations pour Y=1
print(shap_values[...])

Par la suite, on ne regardera que les valeurs de Shapley pour les prédictions $\hat{f}(\mathbf{x}) = \hat{\mathbb{P}}(Y=1 \mid \mathbf{X}= \mathbf{x})$.

In [ ]:
shap_values_class1 = shap_values[...]
print(np.shape(shap_values_class1))

On peut tracer le graphique affichant la moyenne (sur toutes les observations de l'ensemble de test) des valeurs absolues des valeurs SHAP $\phi_j$ pour chaque feature $j \in \{1,\cdots,6\}$ :

In [ ]:
# Importance globale
shap.summary_plot(..., plot_type="bar")

On peut tracer le graphique affichant les valeurs SHAP $\phi_j$ pour chaque feature $j$ et chacune des observations de l'ensemble de test :

In [ ]:
# Importances locales
shap.summary_plot(...)

On peut aussi tracer le graphique de $\phi_1$ (pour la variable "age") pour l'ensemble des observations de la base de test :

In [ ]:
# Importances locales
shap.dependence_plot(..., shap_values_class1, X_df_test)

In [ ]:
# Valeur moyenne de la prédiction de la classe positive sur l'ensemble d'apprentissage
print(...)
# phi_0
print(explainer_shap.expected_value[1])

In [ ]:
# Score prédit pour la première observation (i=0) du jeu de données de test
print(...)
# Classe prédite pour la première observation (i=0) du jeu de données de test
print(...)

In [ ]:
# Valeurs SHAP pour toutes les variables pour l'observation i=0 (pour la classe positive Y=1)
print(...)

In [ ]:
# Vérifier la valeur qu'on obtient en faisant la somme des phi_j et de phi_0
print(... + np.sum(...))

On peut également afficher l'influende de chacune des variables sur la prédiction de la première observation ($i=0$) de la base de test :

In [ ]:
shap.initjs()
shap.force_plot(explainer_shap.expected_value[1], shap_values_class1[0], X_df_test.iloc[0])

On peut aussi s'intéresser à la sixième observation ($i=5$) qui a un score prédit de 0.70 environ.

In [ ]:
shap.force_plot(...)

La fonction "decision_plot" trace l’évolution de la prédiction depuis la valeur de base ($\phi_0$) jusqu’à la prédiction finale en ajoutant la contribution de chaque variable ($\phi_j$) pour les observations spécifiées. On peut tracer ce graphique pour les trois premières observations de l'ensemble de test.

In [ ]:
shap.decision_plot(explainer_shap.expected_value[1], shap_values_class1[:3], X_df_test[...])

## Explication contrefactuelle

In [ ]:
# Import du package
import dice_ml

In [ ]:
# On reforme la base de données d'entraînement en accolant X_train et y_train
df = pd.DataFrame(np.column_stack((X_df_train, y_train)), columns=feature_names + ["Y"])
df[:5]

In [ ]:
# Définition de l'objet DiCE : données
d = dice_ml.Data(dataframe = df, continuous_features = feature_names, outcome_name = ...)
# Définition de l'objet DiCE : modèle
m = dice_ml.Model(model=..., backend='sklearn', model_type='...')

In [ ]:
# Création de l'explainer DiCE
explainer = dice_ml.Dice(..., ..., method="genetic")
# La méthode "genetic" peut gérer les variables catégorielles si besoin

In [ ]:
# Observation de la base de test pour laquelle on veut calculer l'explication contrefactuelle
query_instance = X_df_test.iloc[[0]]
query_instance

In [ ]:
# Affiche la classe prédite pour la première observation de la base de test
print(...)

In [ ]:
# Génération de 3 contrefactuels avec Y=1
# On spécifie les variables qu'on autorise à changer : 'hours_per_week', 'capital_gain' et 'capital_loss'
cf = explainer.generate_counterfactuals(query_instance, total_CFs=3, features_to_vary=...)

# On affiche les explications contrefactuelles obtenues
cf.visualize_as_dataframe(show_only_changes=False)

## ICE (PDP local)

On peut afficher le ICE (ou PDP local) des 10 premières observations de la base d'entraînement pour les variables "age" et "hours_per_week" :

In [ ]:
PartialDependenceDisplay.from_estimator(
    ..., 
    ..., 
    features=..., 
    kind='individual'
    )

## Accumulated Local Effects (ALE)

In [204]:
from PyALE import ale

In [ ]:
# Graphique
ale = ale(model=rf, X=X_df_train, feature=['age'])

In [ ]:
# Affichage du tableau de calcul des effets moyens pour chaque valeur de la variable "age"
ale